In [20]:
import warnings
warnings.filterwarnings('ignore')

In [21]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [22]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

# Read the CSV and Perform Basic Data Cleaning

In [23]:
# https://help.lendingclub.com/hc/en-us/articles/215488038-What-do-the-different-Note-statuses-mean-

columns = [
    "loan_amnt", "int_rate", "installment", "home_ownership",
    "annual_inc", "verification_status", "issue_d", "loan_status",
    "pymnt_plan", "dti", "delinq_2yrs", "inq_last_6mths",
    "open_acc", "pub_rec", "revol_bal", "total_acc",
    "initial_list_status", "out_prncp", "out_prncp_inv", "total_pymnt",
    "total_pymnt_inv", "total_rec_prncp", "total_rec_int", "total_rec_late_fee",
    "recoveries", "collection_recovery_fee", "last_pymnt_amnt", "next_pymnt_d",
    "collections_12_mths_ex_med", "policy_code", "application_type", "acc_now_delinq",
    "tot_coll_amt", "tot_cur_bal", "open_acc_6m", "open_act_il",
    "open_il_12m", "open_il_24m", "mths_since_rcnt_il", "total_bal_il",
    "il_util", "open_rv_12m", "open_rv_24m", "max_bal_bc",
    "all_util", "total_rev_hi_lim", "inq_fi", "total_cu_tl",
    "inq_last_12m", "acc_open_past_24mths", "avg_cur_bal", "bc_open_to_buy",
    "bc_util", "chargeoff_within_12_mths", "delinq_amnt", "mo_sin_old_il_acct",
    "mo_sin_old_rev_tl_op", "mo_sin_rcnt_rev_tl_op", "mo_sin_rcnt_tl", "mort_acc",
    "mths_since_recent_bc", "mths_since_recent_inq", "num_accts_ever_120_pd", "num_actv_bc_tl",
    "num_actv_rev_tl", "num_bc_sats", "num_bc_tl", "num_il_tl",
    "num_op_rev_tl", "num_rev_accts", "num_rev_tl_bal_gt_0",
    "num_sats", "num_tl_120dpd_2m", "num_tl_30dpd", "num_tl_90g_dpd_24m",
    "num_tl_op_past_12m", "pct_tl_nvr_dlq", "percent_bc_gt_75", "pub_rec_bankruptcies",
    "tax_liens", "tot_hi_cred_lim", "total_bal_ex_mort", "total_bc_limit",
    "total_il_high_credit_limit", "hardship_flag", "debt_settlement_flag"
]

target = ["loan_status"]

In [28]:
# Load the data
file_path = Path('LoanStats_2019Q1.csv')
df = pd.read_csv(file_path, skiprows=1)[:-2]
df = df.loc[:, columns].copy()

# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

# Drop the null rows
df = df.dropna()

# Remove the `Issued` loan status
issued_mask = df['loan_status'] != 'Issued'
df = df.loc[issued_mask]

# convert interest rate to numerical
df['int_rate'] = df['int_rate'].str.replace('%', '')
df['int_rate'] = df['int_rate'].astype('float') / 100


# Convert the target column values to low_risk and high_risk based on their values
x = {'Current': 'low_risk'}   
df = df.replace(x)

x = dict.fromkeys(['Late (31-120 days)', 'Late (16-30 days)', 'Default', 'In Grace Period'], 'high_risk')    
df = df.replace(x)

# Adding Code to manualy convert date info to numeric
# Separating the DateParts into different columns runs faster and makes more accurate prediction.
from datetime import datetime

df["issue_d_m"] = df["issue_d"].apply(lambda x: datetime.strptime(x, '%b-%Y').month)
df["issue_d_y"] = df["issue_d"].apply(lambda x: datetime.strptime(x, '%b-%Y').year)
df["next_pymnt_d_m"] = df["next_pymnt_d"].apply(lambda x: datetime.strptime(x, '%b-%Y').month)
df["next_pymnt_d_y"] = df["next_pymnt_d"].apply(lambda x: datetime.strptime(x, '%b-%Y').year)
df.drop('issue_d', axis=1, inplace=True)
df.drop('next_pymnt_d', axis=1, inplace=True)

# Adding Code to create dummies, converting string values into numeric.
df_not_encoded = df
df = pd.get_dummies(df, columns=["home_ownership", 
                                "verification_status", 
                                "pymnt_plan", 
                                "initial_list_status", 
                                "application_type", 
                                "hardship_flag", 
                                "debt_settlement_flag"])

#df.drop(['home_ownership', 'verification_status', 'pymnt_plan', 'initial_list_status', 'application_type', 'hardship_flag', 'debt_settlement_flag'], axis=1, inplace=True)

#-----End Code to convert strings

df.reset_index(inplace=True, drop=True)

df.head()
#df.info()

,loan_amnt,int_rate,installment,annual_inc,loan_status,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,...,verification_status_Not Verified,verification_status_Source Verified,verification_status_Verified,pymnt_plan_n,initial_list_status_f,initial_list_status_w,application_type_Individual,application_type_Joint App,hardship_flag_N,debt_settlement_flag_N
0,10500.0,0.1719,375.35,66000.0,low_risk,27.24,0.0,0.0,8.0,0.0,...,0,1,0,1,0,1,1,0,1,1
1,25000.0,0.2000,929.09,105000.0,low_risk,20.23,0.0,0.0,17.0,1.0,...,0,0,1,1,0,1,1,0,1,1
2,20000.0,0.2000,529.88,56000.0,low_risk,24.26,0.0,0.0,8.0,0.0,...,0,0,1,1,0,1,1,0,1,1
3,10000.0,0.1640,353.55,92000.0,low_risk,31.44,0.0,1.0,10.0,1.0,...,0,0,1,1,0,1,1,0,1,1
4,22000.0,0.1474,520.39,52000.0,low_risk,18.76,0.0,1.0,14.0,0.0,...,1,0,0,1,0,1,1,0,1,1


# Split the Data into Training and Testing

In [29]:
# Create our features
X = df.copy(True)
X.drop('loan_status', axis=1, inplace=True)

# Create our target
# YCH
y = df[["loan_status"]]

In [30]:
X.describe()

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,verification_status_Not Verified,verification_status_Source Verified,verification_status_Verified,pymnt_plan_n,initial_list_status_f,initial_list_status_w,application_type_Individual,application_type_Joint App,hardship_flag_N,debt_settlement_flag_N
count,68817.000000,68817.000000,68817.000000,6.881700e+04,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,...,68817.000000,68817.000000,68817.000000,68817.0,68817.000000,68817.000000,68817.000000,68817.000000,68817.0,68817.0
mean,16677.594562,0.127718,480.652863,8.821371e+04,21.778153,0.217766,0.497697,12.587340,0.126030,17604.142828,...,0.478007,0.373992,0.148001,1.0,0.123879,0.876121,0.860340,0.139660,1.0,1.0
std,10277.348590,0.048130,288.062432,1.155800e+05,20.199244,0.718367,0.758122,6.022869,0.336797,21835.880400,...,0.499520,0.483865,0.355104,0.0,0.329446,0.329446,0.346637,0.346637,0.0,0.0
min,1000.000000,0.060000,30.890000,4.000000e+01,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,1.0,0.000000,0.000000,0.000000,0.000000,1.0,1.0
25%,9000.000000,0.088100,265.730000,5.000000e+04,13.890000,0.000000,0.000000,8.000000,0.000000,6293.000000,...,0.000000,0.000000,0.000000,1.0,0.000000,1.000000,1.000000,0.000000,1.0,1.0
50%,15000.000000,0.118000,404.560000,7.300000e+04,19.760000,0.000000,0.000000,11.000000,0.000000,12068.000000,...,0.000000,0.000000,0.000000,1.0,0.000000,1.000000,1.000000,0.000000,1.0,1.0
75%,24000.000000,0.155700,648.100000,1.040000e+05,26.660000,0.000000,1.000000,16.000000,0.000000,21735.000000,...,1.000000,1.000000,0.000000,1.0,0.000000,1.000000,1.000000,0.000000,1.0,1.0
max,40000.000000,0.308400,1676.230000,8.797500e+06,999.000000,18.000000,5.000000,72.000000,4.000000,587191.000000,...,1.000000,1.000000,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.0,1.0


In [31]:
# Check the balance of our target values
y['loan_status'].value_counts()

low_risk     68470
high_risk      347
Name: loan_status, dtype: int64

In [32]:
from sklearn.model_selection import train_test_split
# YOUR CODE HERE
X_train, X_test, y_train, y_test = train_test_split(X,
    y, random_state=1)

# Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble AdaBoost classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Print the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier onely, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [37]:
# Resample the training data with the BalancedRandomForestClassifier
# YOUR CODE HERE
#inspiration derived from stats.stackexchange.com: Jul 18, 2019 at 2:54 User Jan33
#https://stats.stackexchange.com/questions/417992
#                                      /difference-between-balancedrandomforestclassifier-and-sklearn-ensemble-randomfor

#Adding BalancedRandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier
brf_model = BalancedRandomForestClassifier(n_estimators=100, random_state=1)  #default n_estimators=100

brf_model.fit(X_train, y_train)
#X_resampled, y_resampled = ros.fit_resample(X_train, y_train)

brf_model


BalancedRandomForestClassifier(random_state=1)

In [39]:
# Display the confusion matrix
# YOUR CODE HERE
from sklearn.metrics import confusion_matrix
y_pred = brf_model.predict(X_test)
confusion_matrix(y_test, y_pred)

array([[   64,    37],
       [ 2036, 15068]], dtype=int64)

In [40]:
# Calculated the balanced accuracy score
# YOUR CODE HERE
balanced_accuracy_score(y_test, y_pred)

#---------------------------------------------------------------------------------------------
#pretty sure variance in my numbers vs previous, is due to not having dummied the two date columns: I have Month and Year.
#---------------------------------------------------------------------------------------------

0.7573134418212635

In [41]:
# Print the imbalanced classification report
# YOUR CODE HERE
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.03      0.63      0.88      0.06      0.75      0.54       101
   low_risk       1.00      0.88      0.63      0.94      0.75      0.57     17104

avg / total       0.99      0.88      0.64      0.93      0.75      0.57     17205



In [42]:
# List the features sorted in descending order by feature importance
# YOUR CODE HERE
importances = brf_model.feature_importances_
sorted(zip(importances, X.columns), reverse=True)

[(0.07445097934715064, 'total_pymnt'),
 (0.05973072021834538, 'last_pymnt_amnt'),
 (0.059341930491342, 'total_pymnt_inv'),
 (0.05922302923678781, 'total_rec_int'),
 (0.052037041871302196, 'total_rec_prncp'),
 (0.03134034386688477, 'issue_d_m'),
 (0.031034517912406894, 'int_rate'),
 (0.022257050926400845, 'installment'),
 (0.018220618620553845, 'dti'),
 (0.01739196536474951, 'max_bal_bc'),
 (0.01709604376253827, 'out_prncp_inv'),
 (0.016527166796289988, 'out_prncp'),
 (0.015872351153430598, 'annual_inc'),
 (0.015611699792512626, 'bc_open_to_buy'),
 (0.01540211158591264, 'mths_since_rcnt_il'),
 (0.015005180530685924, 'tot_cur_bal'),
 (0.014740338609157208, 'tot_hi_cred_lim'),
 (0.014612157441914252, 'revol_bal'),
 (0.014464922671268, 'il_util'),
 (0.01442502551090268, 'bc_util'),
 (0.014210427828975724, 'mths_since_recent_inq'),
 (0.0138282539493507, 'total_bal_ex_mort'),
 (0.013726604886879597, 'avg_cur_bal'),
 (0.013674153816010155, 'num_rev_accts'),
 (0.0134730452027174, 'total_bc_lim

### Easy Ensemble AdaBoost Classifier

In [43]:
# Train the EasyEnsembleClassifier
# YOUR CODE HERE
#Adding BalancedRandomForestClassifier
from imblearn.ensemble import EasyEnsembleClassifier
EEC_model = EasyEnsembleClassifier(n_estimators=100, random_state=1)  #default n_estimators=100

EEC_model.fit(X_train, y_train)
#X_resampled, y_resampled = ros.fit_resample(X_train, y_train)

EEC_model


EasyEnsembleClassifier(n_estimators=100, random_state=1)

In [45]:
# Display the confusion matrix
# YOUR CODE HERE
from sklearn.metrics import confusion_matrix
y_pred = EEC_model.predict(X_test)
confusion_matrix(y_test, y_pred)

array([[   93,     8],
       [  961, 16143]], dtype=int64)

In [46]:
# Calculated the balanced accuracy score
# YOUR CODE HERE
balanced_accuracy_score(y_test, y_pred)

0.9323031958247274

In [47]:
# Print the imbalanced classification report
# YOUR CODE HERE
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.09      0.92      0.94      0.16      0.93      0.87       101
   low_risk       1.00      0.94      0.92      0.97      0.93      0.87     17104

avg / total       0.99      0.94      0.92      0.97      0.93      0.87     17205

